<a href="https://colab.research.google.com/github/RafsanJany-44/ACM/blob/master/LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
import numpy as np

data = 'https://raw.githubusercontent.com/RafsanJany-44/Thesis_Project/main/All_DATA/without_sn_and_Epoch/EEG_HMC.csv'
dataset = pd.read_csv(data)
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values


In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [45]:
X_train.shape, X_test.shape

((86760, 75), (21691, 75))

StandardScaler() returns a numpy array. For better understandings we convert that in to pandas dataframe.

In [47]:
X_train2=X_train
X_test2=X_test

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train =sc.fit_transform(X_train)

X_test =sc.fit_transform(X_test)


X_train = pd.DataFrame(X_train, index=X_train2.index, columns=X_train2.columns)
X_test = pd.DataFrame(X_test, index=X_test2.index, columns=X_test2.columns)


**Tuning**

KNN

In [7]:
from sklearn.neighbors import KNeighborsClassifier
KNN_class = KNeighborsClassifier(n_neighbors=7)
KNN_class.fit(X_train, y_train)
y_pred_KNN=KNN_class.predict(X_test)

Radomforest

In [9]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=125, random_state=0)
classifier.fit(X_train, y_train)
y_pred_RF=classifier.predict(X_test)

xgboost

In [10]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(max_depth=14)
xgb_classifier.fit(X_train,y_train)
y_pred_XG = xgb_classifier.predict(X_test)

Accurecy

In [11]:
from sklearn.metrics import accuracy_score
print("Accurecy KNN: ",accuracy_score(y_test, y_pred_KNN))
print("Accurecy RF: ",accuracy_score(y_test, y_pred_RF))
print("Accurecy XG: ",accuracy_score(y_test, y_pred_XG))


Accurecy KNN:  0.7467657660174915
Accurecy RF:  0.7600380005029478
Accurecy XG:  0.7751543770432255
